In [1]:
## imports
# astronomy
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
# plotting
import matplotlib.pyplot as plt
# data 
import numpy as np
import pandas as pd 

# 1. Read Data

In [2]:
# file paths 
path_catalogs = 'C:\\Users\\polar\\OneDrive - The University of Kansas\\AGNerds\\Catalogs'

In [3]:
# open tphot
tphot_data = ascii.read(path_catalogs+'\\tphot.cat')
tphot_cols = tphot_data.colnames

# # show table
# tphot_data.show_in_notebook()
print(tphot_cols)

['field', 'ra', 'dec', 'f560w_uJy', 'f560w_uJy_err', 'f770w_uJy', 'f770w_uJy_err', 'f1000w_uJy', 'f1000w_uJy_err', 'f1280w_uJy', 'f1280w_uJy_err', 'f1500w_uJy', 'f1500w_uJy_err', 'f1800w_uJy', 'f1800w_uJy_err', 'f2100w_uJy', 'f2100w_uJy_err']


In [4]:
# open EGS F22
egs_f22_data = Table.read(path_catalogs+'\\EGS_F22.fits.gz') # get RA/DEC
egs_f22_cols = egs_f22_data.columns

# # print all columns
# for col in egs_f22_cols:
#     print(col)

In [5]:
# open EGS FIR
egs_fir_data = Table.read(path_catalogs+'\\egs_FIR_photometry_catalog.fits')
egs_fir_cols = egs_fir_data.columns

# # print all columns
for col in egs_fir_cols:
    print(col)

ID_iau
ID
RA
Decl
zphot
zp_l68
zp_u68
zspec
Mstar
eMstar
SNR_IR
z_IR
ez_IR
SFR_IR
eSFR_IR
FK
dFK
Fch1
dFch1
Fch2
dFch2
Fch3
dFch3
Fch4
dFch4
F24
dF24
Q24
F100
dF100
Q100
F160
dF160
Q160
F250
dF250
F350
dF350
F450
dF450
F500
dF500
F850_sh
dF850_sh
F850_d
dF850_d
F850
dF850
F1100
dF1100
Q1100
F20cm
dF20cm
xfAGN
xeAGN
xfTOT
xeTOT
xf70
xe70
xf100
xe100
xf160
xe160
xf250
xe250
xf350
xe350
xf450
xe450
xf500
xe500
xf850
xe850
xf1100
xe1100
xf1200
xe1200
xf2000
xe2000
xf2250
xe2250
xf20cm
xe20cm
Ubest
chi2_min
S_chi2_min
R_chi2_min
Type_FIR
Type_SED
Type_AGN


# 2. Match IDs TODO RA and DEC matching, not ID

In [28]:
# get IDs
egs_f22_ID = np.array(egs_f22_data['ID'])[0]
egs_fir_ID = np.array(egs_fir_data['ID']) # use ID_iau  # TODO match by RA and DEC not ID

In [7]:
print('Number of egs_f22:\t',  len(egs_f22_ID))
print('Number of egs_fir:\t',  len(egs_fir_ID))

Number of egs_f22:	 66783
Number of egs_fir:	 41656


In [8]:
# Used to match by id (code from Connor Auge)
def match(a, b):
    b_set = set(b)
    b_match = [i for i, v in enumerate(a) if v in b_set]
    a_set = set(a)
    a_match = [i for i, v in enumerate(b) if v in a_set]
    a_match = np.asarray(a_match)
    b_match = np.asarray(b_match)
    a_match2 = np.argsort(a[b_match])
    b_match2 = np.argsort(b[a_match])
    return b_match[a_match2],a_match[b_match2]

In [9]:
# match 
key_IDf22, key_IDfir = match(egs_f22_ID, egs_fir_ID)

# apply match key
egs_f22_ID_egsMatches = egs_f22_ID[key_IDf22]
egs_fir_ID_egsMatches = egs_fir_ID[key_IDfir]

In [10]:
# test
i=11
print(egs_f22_ID_egsMatches[i])
print(egs_fir_ID_egsMatches[i])

12
12


In [11]:
print('Number of egs_f22 matches:\t',  len(egs_f22_ID_egsMatches))
print('Number of egs_fir matches:\t',  len(egs_fir_ID_egsMatches))

Number of egs_f22 matches:	 39244
Number of egs_fir matches:	 39244


# 3. Match RA and DEC

In [12]:
## get RA and DEC from catalogs

# get RA and DEC from tphot and make array
tphot_RA  = np.array(tphot_data['ra'])
tphot_DEC = np.array(tphot_data['dec'])
# get RA and DEC from egs_f22 
egs_f22_RA  = np.array(egs_f22_data['RA'] )[0][key_IDf22]            # ??? which egs catalog do I get RA and DEC? Are they same? 
egs_f22_DEC = np.array(egs_f22_data['DEC'])[0][key_IDf22]

# get coordinants 
tphot_coord = SkyCoord(ra=tphot_RA*u.deg, dec=tphot_DEC*u.deg)
egs_f22_coord = SkyCoord(ra=egs_f22_RA*u.deg, dec=egs_f22_DEC*u.deg)

In [13]:
print('Number of tphot sources:\t', len(tphot_RA))
print('Number of egs_f22 sources:\t', len(egs_f22_RA))

Number of tphot sources:	 1734
Number of egs_f22 sources:	 39244


In [14]:
## match RA and DEC between catalogs
# DOC: https://docs.astropy.org/en/stable/coordinates/matchsep.html

# idx are indices into catalog that are the closest objects to each of the coordinates in c, 
# d2d are the on-sky distances between them, and 
# d3d are the 3-dimensional distances. 
idx, d2d, d3d = egs_f22_coord.match_to_catalog_sky(tphot_coord) # idx, d2d, d3d = c.match_to_catalog_sky(catalog)

# separation constraint
max_sep = 1.0 * u.arcsec
# max_sep = 0.5 * u.arcsec
sep_constraint = d2d < max_sep  # use on 'c' (egs_f22_coord)
idx_sep = idx[sep_constraint]   # use on 'catalog' (tphot)

# get matches
egs_f22_coord_matches = egs_f22_coord[sep_constraint]
tphot_coord_matches = tphot_coord[idx_sep]

# print length  
print('Number of matches:\t', len(egs_f22_coord_matches))
print('Number of matches:\t', len(tphot_coord_matches))

Number of matches:	 1644
Number of matches:	 1644


In [15]:
# test match
i=12
print(egs_f22_coord_matches[i])
print(tphot_coord_matches[i])

<SkyCoord (ICRS): (ra, dec) in deg
    (215.057096, 52.899535)>
<SkyCoord (ICRS): (ra, dec) in deg
    (215.057116, 52.89952)>


# 4. Find Duplicate RA/DEC Matches

In [16]:
# make mask of unique soruces 
mask = np.zeros(len(idx_sep), dtype=bool)
mask[np.unique(idx_sep, return_index=True)[1]] = True

# get value of duplicates sources 
duplicates = np.unique(idx_sep[~mask])

# set all non-unique sources to False 
for dup in duplicates :
    mask[np.where(idx_sep == dup)] = False

# apply mask to get unique and duplicate sources
idx_sep_unique = idx_sep[mask]
idx_sep_duplicates = idx_sep[~mask]

# print info
print('Number of unique:\t',        len(idx_sep_unique))
print('Number of duplicates:\t',    len(idx_sep_duplicates))    # ??? this is a lot
# print('Duplicate sources:\n',       idx_sep_duplicates)

Number of unique:	 1197
Number of duplicates:	 447


In [17]:
# apply mask to egs sources
egs_f22_i    = np.where(sep_constraint)[0]
egs_f22_i_unique = egs_f22_i[mask]
egs_f22_i_duplicates = egs_f22_i[~mask]

# print info
print('Number of unique:\t', len(egs_f22_i_unique))
print('Number of duplicates:\t', len(egs_f22_i_duplicates))
# print('Duplicates:\n', egs_f22_i_duplicates)

Number of unique:	 1197
Number of duplicates:	 447


In [18]:
# get values 
egs_f22_coord_unique = egs_f22_coord[egs_f22_i_unique]
tphot_coord_unique = tphot_coord[idx_sep_unique]

# test match
i=1
print(egs_f22_coord_unique[i])
print(tphot_coord_unique[i])

<SkyCoord (ICRS): (ra, dec) in deg
    (215.060615, 52.900889)>
<SkyCoord (ICRS): (ra, dec) in deg
    (215.060621, 52.900873)>


In [19]:
# get values 
egs_f22_coord_duplicates = egs_f22_coord[egs_f22_i_duplicates]
tphot_coord_duplicates = tphot_coord[idx_sep_duplicates]

# test match
i=3
print(egs_f22_coord_duplicates[i])
print(tphot_coord_duplicates[i])

<SkyCoord (ICRS): (ra, dec) in deg
    (215.058814, 52.901328)>
<SkyCoord (ICRS): (ra, dec) in deg
    (215.058642, 52.901514)>


# 5. Build Catalog

In [20]:
# helper indexing
tphot_i     = idx_sep_unique
egs_f22_i   = key_IDf22[egs_f22_i_unique]
egs_fir_i   = key_IDfir[egs_f22_i_unique]

# verify that all lengths match
print(len(tphot_i))
print(len(egs_f22_i))
print(len(egs_fir_i))

# save number of matches 
n_matches = len(tphot_i)

1197
1197
1197


In [21]:
print(type(tphot_data))
print(type(egs_f22_data))
print(type(egs_fir_data))

<class 'astropy.table.table.Table'>
<class 'astropy.table.table.Table'>
<class 'astropy.table.table.Table'>


In [22]:
# convert astropy table to pandas dataframe
tphot_df   = tphot_data.to_pandas()
egs_f22_df = egs_f22_data.to_pandas() # ??? busted... probably because *.gz filetype
egs_fir_df = egs_fir_data.to_pandas()


ValueError: Cannot convert a table with multidimensional columns to a pandas DataFrame. Offending columns are: ['ID', 'RA', 'DEC', 'CANDELS_RA', 'CANDELS_DEC', 'F606W', 'F814W', 'F105W', 'F125W', 'F140W', 'F160W', 'F36', 'F45', 'DF606W', 'DF814W', 'DF105W', 'DF125W', 'DF140W', 'DF160W', 'DF36', 'DF45', 'ZA', 'ZL68', 'ZU68', 'ZPEAK', 'ZA_NOIRAC', 'ZL68_NOIRAC', 'ZU68_NOIRAC', 'ZPEAK_NOIRAC']
One can filter out such columns using:
names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
tbl[names].to_pandas(...)